# Import Packages

In [1]:
# Import public modules
import pandas as pd
import os
import sys

# Import custom modules
from foursquare import foursquare as fsqr
from openstreetmap import openstreetmap as osm
from eurostat import germany as ger

# Add path to local pwd file folder
sys.path.insert(0, os.path.abspath(os.environ['USERPROFILE'] + '/Desktop/my_pwd'))
import foursquare_pwd

# Set foursquare credentials
fsqr_client_id = foursquare_pwd.client_id
fsqr_client_secret = foursquare_pwd.client_secret

# Eurostat

In [45]:
# Population_by_age_groups_and_sex
population_by_age_groups_and_sex = ger.Population_by_age_groups_and_sex()
population_by_age_groups_and_sex.api_query_result.shape

(60, 38)

In [46]:
# Population_structure
population_structure = ger.Population_structure()
population_structure.api_query_result.shape

(60, 19)

In [47]:
# Population_by_citizenship
population_by_citizenship = ger.Population_by_citizenship()
population_by_citizenship.api_query_result.shape

(60, 16)

In [48]:
# Living_conditions
living_conditions = ger.Living_conditions()
living_conditions.api_query_result.shape

(60, 19)

In [49]:
# Education
education = ger.Education()
education.api_query_result.shape

(20, 6)

In [50]:
# Culture_and_tourism
culture_and_tourism = ger.Culture_and_tourism()
culture_and_tourism.api_query_result.shape

(60, 11)

In [51]:
# Labour_market
labour_market = ger.Labour_market()
labour_market.api_query_result.shape

(60, 40)

In [52]:
# Transport
transport = ger.Transport()
transport.api_query_result.shape

(60, 11)

In [53]:
# Environment
environment = ger.Environment()
environment.api_query_result.shape

(39, 8)

## Merge Eurostat data into a single DataFrame

In [54]:
eurostat_data = population_by_age_groups_and_sex.api_query_result \
                .merge(population_structure.api_query_result, on=['cities', 'time'], how='outer') \
                .merge(population_by_citizenship.api_query_result, on=['cities', 'time'], how='outer') \
                .merge(living_conditions.api_query_result, on=['cities', 'time'], how='outer') \
                .merge(education.api_query_result, on=['cities', 'time'], how='outer') \
                .merge(culture_and_tourism.api_query_result, on=['cities', 'time'], how='outer') \
                .merge(labour_market.api_query_result, on=['cities', 'time'], how='outer') \
                .merge(transport.api_query_result, on=['cities', 'time'], how='outer') \
                .merge(environment.api_query_result, on=['cities', 'time'], how='outer').drop_duplicates()

In [55]:
print(eurostat_data.shape)
eurostat_data.head()

(60, 152)


,cities,time,"Population on the 1st of January, 0-4 years, female","Population on the 1st of January, 0-4 years, male","Population on the 1st of January, 0-4 years, total","Population on the 1st of January, 10-14 years, female","Population on the 1st of January, 10-14 years, male","Population on the 1st of January, 10-14 years, total","Population on the 1st of January, 15-19 years, female","Population on the 1st of January, 15-19 years, male",...,Number of registered cars per 1000 population,People commuting into the city,People commuting out of the city,People killed in road accidents per 10000 pop.,"Municipal waste generated (domestic and commercial), total - 1000 t",Price of a m³ of domestic water - Euro,Share of population connected to potable drinking water system - %,Share of population connected to sewerage treatment system - %,Share of the urban waste water load (in population equivalents) treated according to the applicable standard -%,Total use of water - m³
0,Berlin,2016,86434.0,91718.0,178152.0,67609.0,71212.0,138821.0,69120.0,72707.0,...,291.2,295838.0,174263.0,0.159,1361.98,1.81,99.81,99.72,100.0,206000000.0
1,Berlin,2017,90989.0,96033.0,187022.0,69626.0,73565.0,143191.0,70952.0,76589.0,...,NaN,309375.0,179977.0,0.101,1385.72,NaN,NaN,NaN,NaN,NaN
2,Berlin,2018,92674.0,97822.0,190496.0,71497.0,75635.0,147132.0,71189.0,76087.0,...,NaN,321219.0,185723.0,0.125,NaN,NaN,NaN,NaN,NaN,NaN
3,Bielefeld,2016,7514.0,7906.0,15420.0,7494.0,8017.0,15511.0,8932.0,9247.0,...,427.2,64648.0,35734.0,0.270,150.47,1.85,98.35,99.85,100.0,15526000.0
4,Bielefeld,2017,7775.0,8112.0,15887.0,7528.0,8054.0,15582.0,8892.0,9346.0,...,NaN,66262.0,37546.0,0.330,152.70,NaN,NaN,NaN,NaN,NaN


# Collect Foursquare data for top 20 cities

## Test biergarten fetch for top 20 cities

In [13]:
osm_fsqr_data = pd.DataFrame()

# Loop through the 20 biggest cities in Germany
for city, city_code in zip(osm.top20_cities.keys(), osm.top20_cities.values()):
    
    # Get biergartens for city from OpenStreetMap
    osm_biergartens = osm.OpenStreetMap(city, city_code)
    osm_biergartens.get_biergartens_for_city()
    
    # Match OpenStreetMap biergartens to Foursquare venues
    venues = fsqr.Venues(fsqr_client_id, fsqr_client_secret)
    venues.get_nearby_venues(osm_biergartens.df_biergartens['city'], osm_biergartens.df_biergartens['latitude'],osm_biergartens.df_biergartens['longitude'])
    
    # Get reviews for venues identified as biergarten
    reviews = fsqr.Reviews(fsqr_client_id, fsqr_client_secret)
    reviews.get_venue_review(venues.venue_df['City'], venues.venue_df['venue_id'])
    
    # Collect results to a single dataframe
    osm_fsqr_data = osm_fsqr_data.append(reviews.review_df)
    

Finding biergartens in Berlin...


0it [00:00, ?it/s]

Found 143 biergartens in Berlin
Matching biergartens in Berlin to Foursquare venues...


143it [02:06,  1.13it/s]
0it [00:00, ?it/s]

Fetching reviews for identified biergartens in Berlin...


...

0it [00:00, ?it/s]

Found 25 biergartens in Mannheim
Matching biergartens in Mannheim to Foursquare venues...


25it [00:21,  1.18it/s]
0it [00:00, ?it/s]

Fetching reviews for identified biergartens in Mannheim...


7it [00:06,  1.13it/s]


In [14]:
print(osm_fsqr_data.shape)
osm_fsqr_data.head()

(426, 5)


,venue_id,venue_name,rating,likes_cnt,city
0,53541e32498e5522be7a5eed,Jockel Biergarten,5.6,44,Berlin
1,4c20c6424889a593e9df2620,Haus Sanssouci,7.7,9,Berlin
2,4adcda73f964a520654521e3,Brauhaus Spandau,8.3,135,Berlin
3,4adcda77f964a520e14521e3,Schleusenkrug,8.5,497,Berlin
4,4adcda74f964a520674521e3,Alter Fritz,7.4,14,Berlin


# Merging Eurostat Data to Biergartens

In [28]:
biergarten_data = osm_fsqr_data.merge(eurostat_data, left_on='city', right_on='cities')

In [39]:
print(biergarten_data.shape)
biergarten_data.head()

(1278, 152)


,venue_id,venue_name,rating,likes_cnt,city,cities,time,"Population on the 1st of January, 0-4 years, female","Population on the 1st of January, 0-4 years, male","Population on the 1st of January, 0-4 years, total",...,Number of private cars registered,Number of registered cars per 1000 population,People commuting into the city,People commuting out of the city,"Municipal waste generated (domestic and commercial), total - 1000 t",Price of a m³ of domestic water - Euro,Share of population connected to potable drinking water system - %,Share of population connected to sewerage treatment system - %,Share of the urban waste water load (in population equivalents) treated according to the applicable standard -%,Total use of water - m³
0,53541e32498e5522be7a5eed,Jockel Biergarten,5.6,44,Berlin,Berlin,2016,86434.0,91718.0,178152.0,...,1024876.0,291.2,295838.0,174263.0,1361.98,1.81,99.81,99.72,100.0,206000000.0
1,53541e32498e5522be7a5eed,Jockel Biergarten,5.6,44,Berlin,Berlin,2017,90989.0,96033.0,187022.0,...,NaN,NaN,309375.0,179977.0,1385.72,NaN,NaN,NaN,NaN,NaN
2,53541e32498e5522be7a5eed,Jockel Biergarten,5.6,44,Berlin,Berlin,2018,92674.0,97822.0,190496.0,...,NaN,NaN,321219.0,185723.0,NaN,NaN,NaN,NaN,NaN,NaN
3,4c20c6424889a593e9df2620,Haus Sanssouci,7.7,9,Berlin,Berlin,2016,86434.0,91718.0,178152.0,...,1024876.0,291.2,295838.0,174263.0,1361.98,1.81,99.81,99.72,100.0,206000000.0
4,4c20c6424889a593e9df2620,Haus Sanssouci,7.7,9,Berlin,Berlin,2017,90989.0,96033.0,187022.0,...,NaN,NaN,309375.0,179977.0,1385.72,NaN,NaN,NaN,NaN,NaN


# Store dataset

In [41]:
biergarten_data.to_csv('Biergarten_data_for_analysis.csv', sep=';', header=True, index=False)